In [7]:
# Install needed libraries
%pip install -U python-jobspy
%pip install tqdm
%pip install xlsxwriter

You should consider upgrading via the '/home/wagner/.pyenv/versions/3.10.0/envs/market_scrapper_venv/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.
You should consider upgrading via the '/home/wagner/.pyenv/versions/3.10.0/envs/market_scrapper_venv/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.
You should consider upgrading via the '/home/wagner/.pyenv/versions/3.10.0/envs/market_scrapper_venv/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [ ]:
from job_scrappers import scrape_glassdoor_jobs
from save_file import setup_output_directory, save_to_json
import os
import pandas as pd

In [9]:
search_terms = ["data scientist", "data analyst", "machine learning engineer"]
locations = ["Argentina", "Remote"]

In [10]:
output_dir = setup_output_directory()
print(f"Output directory: {os.path.abspath(output_dir)}")

Output directory: /home/wagner/Documentos/dev-projects/No Country/Market-Scraper/notebooks/data/raw/jobs_20251113_194134


In [11]:
# Set up output directory
all_jobs = []
    
# Scrape jobs for each search term and location
for term in search_terms:
    for location in locations:
        print(f"\n{'='*50}")
        print(f"Searching for: {term} in {location}")
        print(f"{'='*50}")
            
        try:
            jobs = scrape_glassdoor_jobs(term, location)
            if jobs is not None and not jobs.empty:
                all_jobs.append(jobs)
                # Save individual search results
                save_to_csv(jobs, output_dir)
        except Exception as e:
            print(f"Error scraping {term} in {location}: {str(e)}")
            continue


Searching for: data scientist in Argentina
Scraping 200 data scientist jobs in Argentina...
Saved 62 jobs to data/raw/jobs_20251113_194134/jobs_20251113_194139.csv

Searching for: data scientist in Remote
Scraping 200 data scientist jobs in Remote...
Saved 12 jobs to data/raw/jobs_20251113_194134/jobs_20251113_194141.csv

Searching for: data analyst in Argentina
Scraping 200 data analyst jobs in Argentina...
Saved 181 jobs to data/raw/jobs_20251113_194134/jobs_20251113_194151.csv

Searching for: data analyst in Remote
Scraping 200 data analyst jobs in Remote...
Saved 38 jobs to data/raw/jobs_20251113_194134/jobs_20251113_194155.csv

Searching for: machine learning engineer in Argentina
Scraping 200 machine learning engineer jobs in Argentina...
Saved 36 jobs to data/raw/jobs_20251113_194134/jobs_20251113_194158.csv

Searching for: machine learning engineer in Remote
Scraping 200 machine learning engineer jobs in Remote...
Saved 12 jobs to data/raw/jobs_20251113_194134/jobs_20251113_19

In [12]:
# Combine all jobs and save
if all_jobs:
    combined_jobs = pd.concat(all_jobs, ignore_index=True)
    combined_filename = os.path.join(output_dir, "all_jobs_combined.csv")
    combined_jobs.to_csv(combined_filename, index=False)
    print(f"\nSaved {len(combined_jobs)} total jobs to {combined_filename}")
else:
    print("\nNo jobs were scraped.")


Saved 341 total jobs to data/raw/jobs_20251113_194134/all_jobs_combined.csv


/tmp/ipykernel_181536/299541996.py:3: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  combined_jobs = pd.concat(all_jobs, ignore_index=True)
